In [1]:
import cv2
import numpy as np
import os
import random
from pathlib import Path

# Yolo 로드
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()

# Ensure that net.getUnconnectedOutLayers() returns a list
unconnected_out_layers = net.getUnconnectedOutLayers()
if not isinstance(unconnected_out_layers, list):
    unconnected_out_layers = [unconnected_out_layers]

output_layers = [layer_names[i[0] - 1] for i in unconnected_out_layers]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# 이미지가 있는 디렉토리들의 경로
img_dirs = ["Data/0", "Data/1", "Data/2", "Data/3", "Data/4"]

# 결과 이미지를 저장할 디렉토리 생성
output_dir = Path.home() / "Desktop" / "output_images"
output_dir.mkdir(parents=True, exist_ok=True)

# 랜덤으로 5000장의 이미지를 선택
random_images = []

for img_dir in img_dirs:
    img_files = os.listdir(img_dir)
    random_images.extend(random.sample(img_files, min(5000, len(img_files))))

total_images = len(random_images)  # 총 이미지 개수 추가
detected_images = 0  # 탐지된 이미지 개수 추가

for img_name in random_images:
    for img_dir in img_dirs:
        img_path = os.path.join(img_dir, img_name)
        
        # Ensure the image is successfully loaded
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error: Unable to read image '{img_path}'")
            continue
        
        img = cv2.resize(img, None, fx=0.7, fy=0.7)
        height, width, channels = img.shape

        # Detecting objects
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # 정보를 화면에 표시
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    # 좌표
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        font = cv2.FONT_HERSHEY_PLAIN
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = confidences[i]  # 탐지된 객체의 정확도
                color = colors[i]
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                # Adjust the font scale to change the text size
                font_scale = 1.0
                cv2.putText(img, f"{label} {confidence:.2f}", (x, y + 30), font, font_scale, color, 2)

        # Save the annotated image to the output directory
        output_path = output_dir / f"{img_name}_annotated.jpg"
        cv2.imwrite(str(output_path), img)

        detected_images += 1  # 탐지된 이미지 개수 추가

# 정확도 계산 및 출력
accuracy = detected_images / total_images
print(f"Total Images: {total_images}")
print(f"Detected Images: {detected_images}")
print(f"Accuracy: {accuracy * 100:.2f}%")

print("Annotations saved to", output_dir)


Error: Unable to read image 'Data/1/3371370090.jpg'
Error: Unable to read image 'Data/2/3371370090.jpg'
Error: Unable to read image 'Data/3/3371370090.jpg'
Error: Unable to read image 'Data/4/3371370090.jpg'


[ WARN:0@1.041] global loadsave.cpp:248 findDecoder imread_('Data/1/3371370090.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1.045] global loadsave.cpp:248 findDecoder imread_('Data/2/3371370090.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1.047] global loadsave.cpp:248 findDecoder imread_('Data/3/3371370090.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1.047] global loadsave.cpp:248 findDecoder imread_('Data/4/3371370090.jpg'): can't open/read file: check file path/integrity


Error: Unable to read image 'Data/1/2376922558.jpg'
Error: Unable to read image 'Data/2/2376922558.jpg'
Error: Unable to read image 'Data/3/2376922558.jpg'
Error: Unable to read image 'Data/4/2376922558.jpg'
Error: Unable to read image 'Data/1/2723708108.jpg'
Error: Unable to read image 'Data/2/2723708108.jpg'
Error: Unable to read image 'Data/3/2723708108.jpg'
Error: Unable to read image 'Data/4/2723708108.jpg'


[ WARN:0@1.571] global loadsave.cpp:248 findDecoder imread_('Data/1/2376922558.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1.572] global loadsave.cpp:248 findDecoder imread_('Data/2/2376922558.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1.572] global loadsave.cpp:248 findDecoder imread_('Data/3/2376922558.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1.572] global loadsave.cpp:248 findDecoder imread_('Data/4/2376922558.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1.698] global loadsave.cpp:248 findDecoder imread_('Data/1/2723708108.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1.698] global loadsave.cpp:248 findDecoder imread_('Data/2/2723708108.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1.698] global loadsave.cpp:248 findDecoder imread_('Data/3/2723708108.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1.698] global loadsave.cpp:248 findDecoder imread_('D

Error: Unable to read image 'Data/1/651344573.jpg'
Error: Unable to read image 'Data/2/651344573.jpg'
Error: Unable to read image 'Data/3/651344573.jpg'
Error: Unable to read image 'Data/4/651344573.jpg'


...

Error: Unable to read image 'Data/0/2220864480.jpg'
Error: Unable to read image 'Data/1/2220864480.jpg'
Error: Unable to read image 'Data/2/2220864480.jpg'
Error: Unable to read image 'Data/3/2220864480.jpg'
Error: Unable to read image 'Data/0/300048361.jpg'
Error: Unable to read image 'Data/1/300048361.jpg'
Error: Unable to read image 'Data/2/300048361.jpg'
Error: Unable to read image 'Data/3/300048361.jpg'


[ WARN:0@649.437] global loadsave.cpp:248 findDecoder imread_('Data/0/2220864480.jpg'): can't open/read file: check file path/integrity
[ WARN:0@649.437] global loadsave.cpp:248 findDecoder imread_('Data/1/2220864480.jpg'): can't open/read file: check file path/integrity
[ WARN:0@649.438] global loadsave.cpp:248 findDecoder imread_('Data/2/2220864480.jpg'): can't open/read file: check file path/integrity
[ WARN:0@649.438] global loadsave.cpp:248 findDecoder imread_('Data/3/2220864480.jpg'): can't open/read file: check file path/integrity
[ WARN:0@649.580] global loadsave.cpp:248 findDecoder imread_('Data/0/300048361.jpg'): can't open/read file: check file path/integrity
[ WARN:0@649.580] global loadsave.cpp:248 findDecoder imread_('Data/1/300048361.jpg'): can't open/read file: check file path/integrity
[ WARN:0@649.580] global loadsave.cpp:248 findDecoder imread_('Data/2/300048361.jpg'): can't open/read file: check file path/integrity
[ WARN:0@649.580] global loadsave.cpp:248 findDecod

Error: Unable to read image 'Data/0/94.jpg'
Error: Unable to read image 'Data/1/94.jpg'
Error: Unable to read image 'Data/2/94.jpg'
Error: Unable to read image 'Data/3/94.jpg'
Error: Unable to read image 'Data/0/85.jpg'
Error: Unable to read image 'Data/1/85.jpg'
Error: Unable to read image 'Data/2/85.jpg'
Error: Unable to read image 'Data/3/85.jpg'


[ WARN:0@649.722] global loadsave.cpp:248 findDecoder imread_('Data/0/94.jpg'): can't open/read file: check file path/integrity
[ WARN:0@649.722] global loadsave.cpp:248 findDecoder imread_('Data/1/94.jpg'): can't open/read file: check file path/integrity
[ WARN:0@649.722] global loadsave.cpp:248 findDecoder imread_('Data/2/94.jpg'): can't open/read file: check file path/integrity
[ WARN:0@649.722] global loadsave.cpp:248 findDecoder imread_('Data/3/94.jpg'): can't open/read file: check file path/integrity
[ WARN:0@649.865] global loadsave.cpp:248 findDecoder imread_('Data/0/85.jpg'): can't open/read file: check file path/integrity
[ WARN:0@649.866] global loadsave.cpp:248 findDecoder imread_('Data/1/85.jpg'): can't open/read file: check file path/integrity
[ WARN:0@649.866] global loadsave.cpp:248 findDecoder imread_('Data/2/85.jpg'): can't open/read file: check file path/integrity
[ WARN:0@649.866] global loadsave.cpp:248 findDecoder imread_('Data/3/85.jpg'): can't open/read file: ch

Total Images: 4413
Detected Images: 4508
Accuracy: 102.15%
Annotations saved to /Users/byunheejoo/Desktop/output_images
